In [1]:
import pandas as pd

import os

In [2]:
exclude = ['018_019','018_033','018_038','018_056','018_069','018_089',
           '018_091','018_102','018_117','018_120',
           '018_126','018_128','018_130','018_131',
           'ANON_016','ANON_023','ANON_027','018_011']

In [29]:
late_labels = pd.read_csv(r"D:\extracteddata\latelabels.csv")
early_labels = pd.read_csv(r"D:\extracteddata\newlabels.csv")
parentdircontents = os.listdir(r"D:\extracteddata")
patients = [x for x in parentdircontents if all((os.path.isdir(os.path.join(r"D:\extracteddata",x)),x not in exclude))]
ohe_patient_chars = pd.read_csv(r"D:\H_N\ohe_patient_char.csv")

In [30]:
labelmap = {}
for i,row in late_labels.iterrows():
    if row['OLDID'] not in labelmap.keys():
        labelmap[row['OLDID']] = row['ID']
for i,row in early_labels.iterrows():
    if row['OLDID'] not in labelmap.keys():
        labelmap[row['OLDID']] = row['ID']

In [31]:
patients_new = [pt if pt not in labelmap.keys() else labelmap[pt] for pt in patients]

In [32]:
pt_status = pd.DataFrame(columns=['ID','have_pt_chars','have_label_early','have_label_late'])

In [33]:
pt_status['ID'] = patients_new
pt_status['have_pt_chars'] = pt_status['ID'].apply(lambda x: x in ohe_patient_chars['ANON_ID'].to_list())
pt_status['have_label_early'] = pt_status['ID'].apply(lambda x: x in early_labels['ID'].to_list())
pt_status['have_label_late'] = pt_status['ID'].apply(lambda x: x in late_labels['ID'].to_list())

In [34]:
pt_status = pt_status.join(ohe_patient_chars.set_index("ANON_ID"),on="ID")

In [35]:
early = pt_status[(pt_status['have_pt_chars']==True)&(pt_status['have_label_early']==True)]
late = pt_status[(pt_status['have_pt_chars']==True)&(pt_status['have_label_late']==True)]

In [36]:
early = early.set_index("ID")
late = late.set_index("ID")

In [37]:
early_labels = early_labels.set_index("ID")
late_labels = late_labels.set_index("ID")

In [38]:
early = early.join(early_labels,how="left")
late = late.join(late_labels,how="left")

In [39]:
print('early', early['label'].sum(),len(early))
print('late',late['label'].sum(),len(late))

early 62 87
late 32 74


In [26]:
early.columns

Index(['have_pt_chars', 'have_label_early', 'have_label_late', 'gender_female',
       'gender_male', 'race_am_indian_or_alaska_native', 'race_asian',
       'race_blackaa', 'race_white', 'smoker_current', 'smoker_former',
       'smoker_never', 't_stage_t0', 't_stage_t1', 't_stage_t1a',
       't_stage_t1b', 't_stage_t1c', 't_stage_t2', 't_stage_t2a', 't_stage_t3',
       't_stage_t3a', 't_stage_t3b', 't_stage_t4', 't_stage_t4a',
       't_stage_t4b', 't_stage_tis', 't_stage_tx', 'n_stage_n0', 'n_stage_n1',
       'n_stage_n2', 'n_stage_n2a', 'n_stage_n2b', 'n_stage_n2c', 'n_stage_n3',
       'n_stage_nx', 'm_stage_m0', 'm_stage_m1', 'm_stage_mx', 'hpv_negative',
       'hpv_positive', 'age'],
      dtype='object')